In [ ]:
import os
import json
from collections import Counter

import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch.nn import functional as F
import numpy as np

from estimating_k_dataset import estimating_compactness

num_epoches = 1000

## Get features

In [ ]:
save_dir = "../MCM/features/ImageNet_train_image_features.npy"

if (os.path.isfile(save_dir)):
    image_features = np.load(save_dir)
else:
    import clip
    from tqdm.auto import tqdm
    clip_model, preprocess = clip.load('ViT-B/16', device='cuda', jit=False)
    batch_size = 64

    imageNet_dataset = ImageFolder("/data/qinyu/ImageNet_1k_2012/train/", transform=preprocess)
    imageNet_dataloader = DataLoader(imageNet_dataset, batch_size=batch_size, shuffle=False, num_workers=12, pin_memory=True)
    
    image_features = []
    for images, _ in tqdm(imageNet_dataloader):
        with torch.no_grad():
            image_emb = clip_model.encode_image(images.cuda())
            image_emb = F.normalize(image_emb)
        image_features.append(image_emb.cpu().numpy())
                              
    image_features = np.concatenate(image_features, axis=0)
    np.save(image_features, save_dir)

In [ ]:
text_features = np.load("../MCM/features/test_labels.npy")
image_features.shape, text_features.shape

In [ ]:
imageNet_label = list(json.load(open('./data/descriptors_imagenet.json')).keys())
imageNet_label[:5]

## Whole Dataset

In [ ]:
dataloader = DataLoader(torch.from_numpy(image_features), batch_size=32, shuffle=True)
vmf_class = estimating_compactness(dataloader, max_epochs = num_epoches)
print(f"the compactness k for the whole dataset: {vmf_class.get_params()[1]}")

## Class-wise

In [ ]:
# now we get top 9 classes where most of the images will be assigned into due to the similarity score
assigned_classes = np.argmax(image_features @ text_features.T, axis=1)
counts = Counter(assigned_classes)
sorted_dict = [idx for idx, num in counts.most_common(10)]
sorted_dict

In [ ]:
# now obtain the compacteness
for label in list(sorted_dict):
    embed_label = image_features[assigned_classes == label]
    
    class_dataloader = DataLoader(torch.from_numpy(embed_label), batch_size=32, shuffle=True)

    vmf_class = estimating_compactness(class_dataloader, max_epochs = num_epoches)
    print(f"the compactness k for class {imageNet_label[int(label)]} is {vmf_class.get_params()[1]}")